In [7]:
from PIL import Image

def rotate_and_scale(filename):
  src =  Image.open(f'../source/images/{filename}.png')
  for i in range(-127,127):
    angle = 90*i/128
    dst = src.rotate(angle, resample=Image.NEAREST, expand=True)
    dst = dst.crop(dst.getbbox())
    dst.save(f'../source/images/{filename}_{i}.png',bits=1,optimize=False)

In [8]:
for filename in ["pine_snow_0"]:
  rotate_and_scale(filename)